In [15]:
#必要なライブラリ、定数
import sys
sys.path.append("../channel")
from AWGN import _AWGN
import ray
import numpy as np
import random
import time
import math
from decimal import *
random.seed(time.time())

ch=_AWGN()

In [4]:
class coding():
    def __init__(self,N):
        super().__init__()

        self.N=N
        self.R=0.5
        self.K=math.floor(self.R*self.N)
        self.design_SNR=4

        #prepere constants
        tmp2=np.log2(self.N)
        self.itr_num=tmp2.astype(int)
        self.frozen_bits,self.info_bits=\
            self.Bhattacharyya_bounds()

        self.Gres=self.make_H()

        self.filename="polar_code_{}_{}".format(self.N,self.K)

In [16]:
class coding(coding):
  #frozen_bitの選択
  def Bhattacharyya_bounds(self):
    E=np.zeros(1,dtype=np.float128)
    E =Decimal('10') ** (Decimal(str(self.design_SNR)) / Decimal('10'))
    itr_num=np.log2(N)
    itr_num=itr_num.astype(int)
    z=np.zeros(self.N,dtype=np.float128)

    #10^10かけて計算する

    z[0]=math.exp(Decimal('-1')*Decimal(str(E)))

    #print("E=",np.exp(-E))

    for j in range(1,itr_num+1):
      tmp=2**(j)//2

      for t in range(tmp):
        T=z[t]
        z[t]=Decimal('2')*Decimal(str(T))-Decimal(str(T))**Decimal('2')
        z[tmp+t]=Decimal(str(T))**Decimal('2')
    #print(z)
    #np.savetxt("z",z)
    tmp=self.indices_of_elements(z,N)
    frozen_bits=tmp[:self.N-self.K]
    info_bits=tmp[self.N-self.K:]
    return np.sort(frozen_bits),np.sort(info_bits)

  @staticmethod
  def indices_of_elements(v,l):
    tmp=np.argsort(v)[::-1]
    #print(tmp)
    res=tmp[0:l]
    return res

SyntaxError: invalid syntax (<ipython-input-16-7ba98a416e19>, line 1)

In [6]:
class coding(coding):
  @staticmethod
  def tensordot(A):
    tmp0=np.zeros((A.shape[0],A.shape[1]),dtype=np.int)
    tmp1=np.append(A,tmp0,axis=1)
    #print(tmp1)
    tmp2=np.append(A,A,axis=1)
    #print(tmp2)
    tmp3=np.append(tmp1,tmp2,axis=0)
    #print(tmp3)
    return tmp3

  def make_H(self):
    G2=np.array([[1,0],[1,1]],dtype=np.int)
    Gres=G2
    for _ in range(self.itr_num-1):
      #print(i)
      Gres=self.tensordot(Gres)
    return Gres

In [7]:
class encoding(coding):
    def __init__(self,N):
        super().__init__(N)
    
    def generate_information(self):
        #generate information
        information=np.random.randint(0,2,self.K)
        return information

In [17]:
class encoding(encoding):
    def generate_U(self,information):
        u_message=np.zeros(self.N)
        u_message[self.info_bits]=information
        return u_message

In [18]:
class encoding(encoding):
    def polar_encode(self):
        information=self.generate_information()
        u_message=self.generate_U(information)
        codeword=(u_message@self.Gres)%2
        return information,codeword

In [10]:
#これを使ったほうが計算が早い
'''
%%add_to encoding
def encode(self,u_message):
    """
    Implements the polar transform on the given message in a recursive way (defined in Arikan's paper).
    :param u_message: An integer array of N bits which are to be transformed;
    :return: codedword -- result of the polar transform.
    """
    u_message = np.array(u_message)

    if len(u_message) == 1:
        codeword = u_message
    else:
        u1u2 = np.logical_xor(u_message[::2] , u_message[1::2])
        u2 = u_message[1::2]

        codeword = np.concatenate([encode(u1u2), encode(u2)])
    return codeword
'''

'\n%%add_to encoding\ndef encode(self,u_message):\n    """\n    Implements the polar transform on the given message in a recursive way (defined in Arikan\'s paper).\n    :param u_message: An integer array of N bits which are to be transformed;\n    :return: codedword -- result of the polar transform.\n    """\n    u_message = np.array(u_message)\n\n    if len(u_message) == 1:\n        codeword = u_message\n    else:\n        u1u2 = np.logical_xor(u_message[::2] , u_message[1::2])\n        u2 = u_message[1::2]\n\n        codeword = np.concatenate([encode(u1u2), encode(u2)])\n    return codeword\n'

In [11]:
#0,1が逆になって設計されているので、ちゃんと治す必要あり

class decoding(coding):
  n=0
  EST_information=np.array([])

  def __init__(self,N):
    super().__init__(N)

  @staticmethod
  def chk(llr_1,llr_2):
    CHECK_NODE_TANH_THRES=30
    res=np.zeros(len(llr_1))
    for i in range(len(res)):

      if abs(llr_1[i]) > CHECK_NODE_TANH_THRES and abs(llr_2[i]) > CHECK_NODE_TANH_THRES:
        if llr_1[i] * llr_2[i] > 0:
          # If both LLRs are of one sign, we return the minimum of their absolute values.
          res[i]=min(abs(llr_1[i]), abs(llr_2[i]))
        else:
          # Otherwise, we return an opposite to the minimum of their absolute values.
          res[i]=-1 * min(abs(llr_1[i]), abs(llr_2[i]))
      else:
        res[i]= 2 * np.arctanh(np.tanh(llr_1[i] / 2, ) * np.tanh(llr_2[i] / 2))
    return res

  def SC_decoding(self,a):
    #interior node operation
    if a.shape[0]==1:
      #frozen_bit or not
      if np.any(self.frozen_bits==decoding.n):
        tmp0=np.zeros(1)
      elif a>=0:
        tmp0=np.zeros(1)
      elif a<0:
        tmp0=np.ones(1)
      else:
        print("err!")
        exit()
      
      if np.any(self.info_bits==decoding.n):
        decoding.EST_information=np.append(decoding.EST_information,a)
      #print(decoding.n)
      #print(t)
      decoding.n+=1
      #if t>=N:
        #exit()
      return tmp0

    #step1 left input a output u1_hat

    tmp1=np.split(a,2)
    f_half_a=self.chk(tmp1[0],tmp1[1])
    u1=self.SC_decoding(f_half_a)

    #step2 right input a,u1_hat output u2_hat 
    tmp2=np.split(a,2)
    g_half_a=tmp2[1]+(1-2*u1)*tmp2[0] 
    u2=self.SC_decoding(g_half_a)
  
    #step3 up input u1,u2 output a_hat
    res=np.concatenate([(u1+u2)%2,u2])
    return res

In [19]:
class decoding(decoding):
    def polar_decode(self,Lc):
        #initialize class variable
        decoding.n=0
        decoding.EST_information=np.array([])
        self.SC_decoding(Lc)
        res=decoding.EST_information
        res=-1*np.sign(res)
        EST_information=(res+1)/2

        return EST_information

In [20]:
#@ray.remote
class polar_code(encoding,decoding):
    def __init__(self,N):
        super().__init__(N)

    def polar_code(self,EbNodB):
        

        information,codeword=self.polar_encode()

        Lc=-1*ch.generate_LLR(codeword,EbNodB)#デコーダが＋、ー逆になってしまうので-１をかける
        
        EST_information=self.polar_decode(Lc)
        
        return information,EST_information

In [21]:
if __name__=="__main__":
    
    N=512
    pc=polar_code(N)
    
    print(len(pc.info_bits))
    information,EST_information=pc.polar_code(100)
    print(len(information))
    print(len(EST_information))
    print(np.sum(information!=EST_information))

    def output(EbNodB):
      count_err=0
      count_all=0
      count_berr=0
      count_ball=0
      MAX_ERR=8

      while count_err<MAX_ERR:

        information,EST_information=pc.polar_code(EbNodB)
      
        if np.any(information!=EST_information):#BLOCK error check
          count_err+=1
        
        count_all+=1

        #calculate bit error rate 
        count_berr+=np.sum(information!=EST_information)
        count_ball+=N

        print("\r","count_all=",count_all,",count_err=",count_err,"count_ball="\
              ,count_ball,"count_berr=",count_berr,end="")

      print("\n")
      print("BER=",count_berr/count_ball)
    
    for EbNodB in range(-3,5):
      output(EbNodB)

256
256
256
0
 count_all= 8 ,count_err= 8 count_ball= 4096 count_berr= 996

BER= 0.2431640625
 count_all= 8 ,count_err= 8 count_ball= 4096 count_berr= 960

BER= 0.234375
 count_all= 8 ,count_err= 8 count_ball= 4096 count_berr= 1014

BER= 0.24755859375
 count_all= 8 ,count_err= 8 count_ball= 4096 count_berr= 944

BER= 0.23046875
 count_all= 8 ,count_err= 8 count_ball= 4096 count_berr= 636

BER= 0.1552734375
 count_all= 5 ,count_err= 3 count_ball= 2560 count_berr= 176

KeyboardInterrupt: 